### Performance reports

In [ ]:
# For secrets
from secret import rpc_user, rpc_password

In [ ]:
## Check the bitcoind running
import os

datadir = os.path.abspath(os.path.expanduser('~/.bitcoin'))
pid_path = os.path.join(datadir, 'bitcoind.pid')
if os.path.exists(pid_path):
    with open(pid_path, 'r') as f:
        print(f'Bitcoind PID: {f.read()}')
else:
    raise SystemExit('Bitcoind is not running!')

In [ ]:
## Some library
from address_convertor import pubkey_to_address, get_pubkey

In [ ]:
## Some function for convenient
import datetime

tz_seoul = datetime.timezone(datetime.timedelta(hours=9))
tz_utc = datetime.timezone(datetime.timedelta())

def get_time(timestamp):
    return datetime.datetime.fromtimestamp(timestamp, tz=tz_seoul)

In [ ]:
# preparation database
import sqlite3

db_path = 'index.db'
conn = sqlite3.connect(db_path)
cur = conn.cursor()

def create_blkid_table():
    cur.execute('''CREATE TABLE IF NOT EXISTS BlkID (
                     id INTEGER PRIMARY KEY,
                     blkhash TEXT NOT NULL UNIQUE);
                ''')

def create_txid_table():
    cur.execute('''CREATE TABLE IF NOT EXISTS TxID (
                     id INTEGER PRIMARY KEY,
                     txhash TEXT NOT NULL UNIQUE);
                ''')

def create_addrid_table():
    cur.execute('''CREATE TABLE IF NOT EXISTS AddrID (
                     id INTEGER PRIMARY KEY,
                     addr TEXT NOT NULL UNIQUE);
                ''')

def insert_blkid(blkhash):
    cur.execute('''INSERT OR IGNORE INTO BlkID (
                       blkhash) VALUES (
                       ?);
                    ''', (blkhash,))

def insert_blkids(blkhash):
    cur.executemany('''INSERT OR IGNORE INTO BlkID (
                       blkhash) VALUES (
                       ?);
                    ''', blkhash)
    
def insert_txid(txhash):
    cur.execute('''INSERT OR IGNORE INTO TxID (
                       txhash) VALUES (
                       ?);
                    ''', (txhash,))

def insert_txids(txhash):
    cur.executemany('''INSERT OR IGNORE INTO TxID (
                       txhash) VALUES (
                       ?);
                    ''', txhash)

def insert_addrid(addr):
    cur.execute('''INSERT OR IGNORE INTO AddrID (
                       addr) VALUES (
                       ?);
                    ''', (addr,))

def insert_addrids(addr):
    cur.executemany('''INSERT OR IGNORE INTO AddrID (
                       addr) VALUES (
                       ?);
                    ''', addr)
    
def begin_transactions():
    cur.execute('BEGIN TRANSACTION;')

def commit_transactions():
    cur.execute('COMMIT;')

def journal_mode(mode):
    sql = f'PRAGMA journal_mode = {mode}'
    cur.execute(sql)
    conn.commit()

def get_max_height():
    cur.execute('''SELECT MAX(id) FROM BlkID;''')
    return cur.fetchone()[0] 

In [ ]:
from bitcoinrpc.authproxy import AuthServiceProxy, JSONRPCException
rpc_ip = '127.0.0.1'
rpc_port = '8332'
timeout = 60*5

def get_rpc():
    return AuthServiceProxy(f'http://{rpc_user}:{rpc_password}@{rpc_ip}:{rpc_port}', timeout=timeout)

rpc_connection = get_rpc()
best_block_hash = rpc_connection.getbestblockhash()
best_block = rpc_connection.getblock(best_block_hash)
print(f'Best Block Heights: {best_block["height"]}, Time: {get_time(best_block["time"]).isoformat()}')

In [ ]:
create_blkid_table()
create_txid_table()
create_addrid_table()

In [ ]:
def get_data(theight):
    blks = list()
    txes = list()
    addrs = list()
    block_hash = rpc_connection.getblockhash(theight)
    block = rpc_connection.getblock(block_hash, 2)
    blks.append(block_hash)
    for tx in block['tx']:
        txes.append(tx['txid']) # IMPORTRANT! We should to use txid not hash
        n = -1
        for vout in tx['vout']:
            n = n + 1
            if vout['scriptPubKey']['type'] in ('pubkeyhash', 'scripthash', 
                                                'witness_v0_keyhash', 'witness_v0_scripthash',
                                                'witness_unknown', 'multisig'):
                addrs.extend(vout['scriptPubKey']['addresses'])
            elif vout['scriptPubKey']['type'] in ('pubkey', 'nonstandard', 'nulldata'):
                try:
                    addrs.extend([pubkey_to_address(get_pubkey(vout['scriptPubKey']['hex']))])
                except UnboundLocalError:
                    addrs.extend([f'{tx["txid"]}{n}'])
            else:
                raise SystemExit(f'BUG!! {theight}\n{tx}\n{vout}')
    return blks, txes, addrs

In [ ]:
import time
import pickle
import itertools
import multiprocessing

if os.path.exists('taking_index.pickle'):
    with open('taking_index.pickle', 'rb') as f:
        taking = pickle.load(f)
else:
    taking = dict()

term = 1000
start_height = get_max_height()
if start_height is None or start_height < term:
    start_height = 0
else:
    start_height = (start_height//term-1)*term
end_height = (best_block['height']//term-1)*term # default
print(f'Start from {start_height} to {end_height}')

rpc_connection = get_rpc()
pool_num = multiprocessing.cpu_count()//2
try:
    journal_mode('WAL')
    for sheight, eheight in zip(range(start_height, end_height, term), 
                                range(start_height+term, end_height+term, term)):
        stime = time.time()
        begin_transactions()
        if eheight >= end_height:
            eheight = end_height+1
        with multiprocessing.Pool(pool_num) as p:
            try:
                results = p.imap(get_data, range(sheight, eheight))
                for blks, txes, addrs in results:
                    for blk in blks:
                        insert_blkid(blk)
                    for tx in txes:
                        insert_txid(tx)
                    for addr in addrs:
                        insert_addrid(addr)
            except KeyboardInterrupt:
                print(f'KeyboardInterrupt occured. Send terminate to processes')
                p.terminate()
                p.join()
                raise KeyboardInterrupt
        commit_transactions()
        etime = time.time()
        taking[sheight] = etime-stime
        print(f'Job done from {sheight} to {eheight-1} during {etime-stime}')
except KeyboardInterrupt:
    print(f'KeyboardInterrupt detected. Commit transactions.')
    commit_transactions()
finally:
    try:
        commit_transactions()
    except sqlite3.OperationalError:
        pass
    journal_mode('DELETE')
    with open('taking_index.pickle', 'wb') as f:
        pickle.dump(taking, f)
    conn.close()